In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_csv(("../data/cleaned/hcmhouseprice.csv"))

df_processed = df.drop('address', axis=1)
df_processed = df_processed.dropna()

feature_cols = ['area', 'bedrooms', 'bathrooms']
target_col = 'price'

X_df = df_processed[feature_cols]
y_series = df_processed[target_col]

X_train_df, X_test_df, y_train_series, y_test_series = train_test_split(
    X_df, y_series, test_size=0.2, random_state=42
)

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_df)
X_test_scaled = scaler_X.transform(X_test_df)

scaler_Y = StandardScaler()
y_train_scaled = scaler_Y.fit_transform(y_train_series.values.reshape(-1, 1))
y_test_scaled = scaler_Y.transform(y_test_series.values.reshape(-1, 1))

X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train_scaled, dtype=torch.float32) 

X_test = torch.tensor(X_test_scaled, dtype=torch.float32)

In [3]:
model = torch.nn.Linear(in_features=X_train.shape[1], out_features=1)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10000
for epoch in range(epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")


Epoch 1000/10000, Loss: 0.771086
Epoch 2000/10000, Loss: 0.771054
Epoch 3000/10000, Loss: 0.771054
Epoch 4000/10000, Loss: 0.771054
Epoch 5000/10000, Loss: 0.771054
Epoch 6000/10000, Loss: 0.771054
Epoch 7000/10000, Loss: 0.771054
Epoch 8000/10000, Loss: 0.771054
Epoch 9000/10000, Loss: 0.771054
Epoch 10000/10000, Loss: 0.771054


In [4]:
model.eval() 

with torch.no_grad():
    y_pred_scaled_tensor = model(X_test)
    y_pred_scaled = y_pred_scaled_tensor.numpy()
    
    y_pred = scaler_Y.inverse_transform(y_pred_scaled)
    
    y_true = y_test_series.values.flatten()
    
    r2 = r2_score(y_true, y_pred.flatten())
    mae = mean_absolute_error(y_true, y_pred.flatten())
    rmse = np.sqrt(mean_squared_error(y_true, y_pred.flatten()))
        
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f} (tỷ VNĐ)")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f} (tỷ VNĐ)")

R-squared (R²): -364.0241
Mean Absolute Error (MAE): 17.0210 (tỷ VNĐ)
Root Mean Squared Error (RMSE): 254.5751 (tỷ VNĐ)


In [5]:
w = model.weight.data.numpy().flatten()
b = model.bias.item()

for name, weight in zip(feature_cols, w):
    print(f"{name:25s}: {weight: .4f}")

print(f"\nBias (intercept): {b:.4f}")

area                     :  0.5208
bedrooms                 :  0.0109
bathrooms                : -0.0769

Bias (intercept): 0.0000
